In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 28768697
paper_name = 'zacchi_bernstein_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/TableS1-S6.xlsx', sheet_name='ST-1', skiprows=10)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 2851 x 11


In [7]:
original_data.head()

,Plate,Row,Column,Original Signal Value,Normalized Value,Log2 Normalized value,Z-Scores,P-Value,ORF,Gene,Description
0,[4],I,5,45902.86,3.454527,1.788488,6.605890,3.951400e-11,YKL008C,LAC1,Ceramide synthase component; involved in synth...
1,[3],M,5,77489.63,3.341991,1.740708,6.294210,3.089800e-10,YER065C,ICL1,Isocitrate lyase; catalyzes the formation of s...
2,[1],J,11,71690.99,3.304119,1.724266,6.189318,6.042400e-10,YLR073C,RFU1,Protein that inhibits Doa4p deubiquitinating a...
3,[2],K,11,41766.77,3.125455,1.644066,5.694488,1.237420e-08,YPL201C,YIG1,Protein that interacts with glycerol 3-phospha...
4,[2],F,9,55963.26,3.117048,1.640180,5.671203,1.417980e-08,YOR049C,RSB1,Suppressor of sphingoid LCB sensitivity of an ...


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Plate, Row, Column, Original Signal Value, Normalized Value, Log2 Normalized value, Z-Scores, P-Value, ORF, Gene, Description]
Index: []


In [12]:
original_data['data'] = original_data['Log2 Normalized value']

In [13]:
original_data.set_index('ORF', inplace=True)

# Prepare the final dataset

In [14]:
dataset_ids = [16445]

In [15]:
datasets = datasets.reindex(index=dataset_ids)

In [16]:
data = original_data['data'].to_frame()

In [17]:
data.columns = datasets['name'].values

In [19]:
data = data.groupby(data.index).mean()

In [20]:
# Create row index
data.index.name='orf'

In [21]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 2826 x 1


# Print out

In [22]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [23]:
from IO.save_data_to_db2 import *

In [24]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [25]:
save_data_to_db(data, paper_pmid)